In [1]:
import os

# Prepend the folder with espeak.exe to PATH
os.environ["PATH"] = r"C:\Program Files (x86)\eSpeak\command_line\command_line;" + os.environ["PATH"]


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate

from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1026)
           2	LOAD_FAST(arg=0, lineno=1026)
           4	LOAD_CONST(arg=1, lineno=1026)
           6	BINARY_SUBSCR(arg=None, lineno=1026)
           8	LOAD_FAST(arg=0, lineno=1026)
          10	LOAD_CONST(arg=2, lineno=1026)
          12	BINARY_SUBSCR(arg=None, lineno=1026)
          14	COMPARE_OP(arg=4, lineno=1026)
          16	LOAD_FAST(arg=0, lineno=1026)
          18	LOAD_CONST(arg=1, lineno=1026)
          20	BINARY_SUBSCR(arg=None, lineno=1026)
          22	LOAD_FAST(arg=0, lineno=1026)
          24	LOAD_CONST(arg=3, lineno=1026)
          26	BINARY_SUBSCR(arg=None, lineno=1026)
          28	COMPARE_OP(arg=5, lineno=1026)
          30	BINARY_AND(arg=None, lineno=1026)
          32	RETURN_VALUE(arg=None, lineno=1026)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=102

## LJ Speech

In [3]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [4]:
import torch
print(torch.cuda.is_available())


False


In [5]:
"""net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()
"""
"""_ = utils.load_checkpoint("/path/to/pretrained_ljs.pth", net_g, None)"""

import sys
sys.path.append(r"C:\Users\MIKKY\vits-main\vits-main")

import torch
from models import SynthesizerTrn
import utils

device = "cpu"

# Initialize model
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
)
net_g.to(device)
net_g.eval()

# Load checkpoint (CPU-only)
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_ljs.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])  # Use 'model' key

print("Model loaded successfully on CPU!")



C:\Users\MIKKY\AppData\Roaming\Python\Python38\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\MIKKY\AppData\Local\Temp\ipykernel_19860\2145425827.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you

Model loaded successfully on CPU!


In [6]:
import os
os.system("espeak --version")


0

In [8]:
"""import os

# Prepend the folder that contains espeak.exe to PATH
os.environ["PATH"] = r"C:\Program Files (x86)\eSpeak\command_line\command_line;" + os.environ["PATH"]


"""

'import os\n\n# Prepend the folder that contains espeak.exe to PATH\nos.environ["PATH"] = r"C:\\Program Files (x86)\\eSpeak\\command_line\\command_line;" + os.environ["PATH"]\n\n\n'

In [7]:
net_g.to(device)
net_g.eval()
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])  # use 'model' key in checkpoint


C:\Users\MIKKY\AppData\Local\Temp\ipykernel_19860\1787058031.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

<All keys matched successfully>

In [44]:
import os
import torch
from models import SynthesizerTrn
import utils
from text import text_to_sequence
from text.symbols import symbols   # ✅ import here
import scipy.io.wavfile as wavfile

# 1️⃣ Use CPU
device = "cpu"

# 2️⃣ Load configs
hps = utils.get_hparams_from_file("configs/ljs_base.json")

# 3️⃣ Initialize model
net_g = SynthesizerTrn(
    len(symbols),   # ✅ use imported symbols
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
)
net_g.to(device)
net_g.eval()

# 4️⃣ Load pretrained checkpoint
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_ljs.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])

# 5️⃣ Convert text → IDs
text = "VITS is Awesome!"
stn_tst = torch.LongTensor(
    text_to_sequence(text, hps.data.text_cleaners)  # ✅ only 2 args
)


# 6️⃣ Prepare input
x_tst = stn_tst.unsqueeze(0).to(device)
x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

# 7️⃣ Inference
with torch.no_grad():
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().numpy()

# 8️⃣ Save to wav
wavfile.write("output.wav", hps.data.sampling_rate, audio.astype('float32'))
print("✅ Done! Audio saved as output.wav")


C:\Users\MIKKY\AppData\Roaming\Python\Python38\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\MIKKY\AppData\Local\Temp\ipykernel_19860\2637349133.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you

✅ Done! Audio saved as output.wav


In [54]:
from phonemizer import phonemize

text = "Good evening"
print(phonemize(text, language='en-us', backend='espeak'))


ɡʊd iːvnɪŋ 


In [ ]:
"""stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))"""
# ✅ CPU-safe inference (original)

# 1️⃣ Convert text → sequence
stn_tst = get_text("Good evening everyone! VITS is Awesome!", hps)

# 2️⃣ Inference
with torch.no_grad():
    # send to CPU
    x_tst = stn_tst.unsqueeze(0).to(device)  
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

    # run model
    
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().numpy()

# 3️⃣ Play or save audio
import IPython.display as ipd
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [ ]:
#approach A rhythm division according to phonemes- failed 
def infer_with_stress(self, x, x_lengths, sid=None, noise_scale=1.0, length_scale=1.0,
                      noise_scale_w=1.0, stress_indices=None, stress_length_scale=1.0):
    """
    VITS inference with optional stress applied to selected phonemes.

    Args:
        x: Input phoneme sequence [B, T]
        x_lengths: Lengths of input sequences
        sid: Speaker ID (optional)
        noise_scale: Noise scale for latent z
        length_scale: Global length scaling
        noise_scale_w: Noise scale for duration predictor
        stress_indices: List of phoneme indices to stress
        stress_length_scale: Factor to exaggerate durations of stress_indices

    Returns:
        audio: Generated waveform [B, 1, T]
        m_p, logs_p, None, None, attn: VITS-compatible return values
    """


    """
    # --- Encoder ---
    x, m_p, logs_p, x_mask = self.enc_p(x, x_lengths)

    # --- Duration prediction ---
    logw = self.dp(x, x_mask, reverse=True, noise_scale=noise_scale_w)  # log duration
    w = torch.exp(logw) * x_mask  # durations

    # --- Apply stress scaling only to selected indices ---
    if stress_indices is not None and stress_length_scale != 1.0:
        for idx in stress_indices:
            if idx < w.size(2):
                w[:, :, idx] *= stress_length_scale

    # --- Apply global length scale ---
    w_ceil = torch.ceil(w) * length_scale

    # --- Compute output mask ---
    y_lengths = torch.clamp_min(torch.sum(w_ceil, [1, 2]), 1).long()
    y_mask = torch.unsqueeze(commons.sequence_mask(y_lengths, None), 1).to(x_mask.dtype)

    # --- Generate attention/alignment ---
    attn_mask = torch.unsqueeze(x_mask, 2) * torch.unsqueeze(y_mask, -1)
    attn = commons.generate_path(w_ceil, attn_mask)

    # --- Project prior to latent space ---
    m_p = torch.matmul(attn.squeeze(1), m_p.transpose(1, 2)).transpose(1, 2)
    logs_p = torch.matmul(attn.squeeze(1), logs_p.transpose(1, 2)).transpose(1, 2)

    # --- Latent sampling ---
    z_p = (m_p + torch.randn_like(m_p) * torch.exp(logs_p) * noise_scale) * y_mask
    z = self.flow(z_p, y_mask, reverse=True)

    # --- Decoder ---
    audio = self.dec(z * y_mask)

    # --- Return VITS-compatible outputs ---
    return audio, m_p, logs_p, None, None, attn
"""

In [ ]:
"""from phonemizer import phonemize

text = "Good evening! VITS is awesome!"
phonemes_text = phonemize(text, language='en-us', backend='espeak')
print(phonemes_text)
phonemes_list = phonemes_text.split()  # ['ɡʊd', 'iːvnɪŋ', 'vɪts', 'ɪz', 'ɔːsʌm']
print(phonemes_list)"""


ɡʊd iːvnɪŋ vɪts ɪz ɔːsʌm 
['ɡʊd', 'iːvnɪŋ', 'vɪts', 'ɪz', 'ɔːsʌm']


In [ ]:
"""word = "awesome"
word_phonemes = phonemize(word, language='en-us', backend='espeak').split()
print(word_phonemes)  # ['ɔːsʌm']
"""

['ɔːsʌm']


In [ ]:
"""def find_phoneme_indices(sentence_phonemes, word_phonemes):
    for i in range(len(sentence_phonemes) - len(word_phonemes) + 1):
        if sentence_phonemes[i:i+len(word_phonemes)] == word_phonemes:
            return list(range(i, i+len(word_phonemes)))
    return []

stress_indices = find_phoneme_indices(phonemes_list, word_phonemes)
print(stress_indices) """


[4]


In [ ]:
"""# define the stress wrapper outside models.py
from types import MethodType

net_g.infer_with_stress = MethodType(infer_with_stress, net_g)

with torch.no_grad():
    audio = net_g.infer_with_stress(
        x_tst,
        x_tst_lengths,
        noise_scale = 1,
        noise_scale_w= 2,
        length_scale=1,
        stress_indices= stress_indices,   # positions for "awesome"
        stress_length_scale= 2
    )[0][0,0].cpu().numpy()

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
"""

going with option B - Synthesize Stress Word Separately (Successful)



In [ ]:
# going with option A:
text = "Good evening! VITS is awesome!"
stress_word = "awesome"

before_word = text.split(stress_word)[0].strip()  # "Good evening! VITS is"
after_word = text.split(stress_word)[1].strip()   # "!" (or remaining words)


In [91]:
# Whole sentence → phonemes
stn_before = get_text(before_word, hps)
stn_stress = get_text(stress_word, hps)
stn_after = get_text(after_word, hps) if after_word else None

x_lengths_before = torch.LongTensor([stn_before.size(0)])
x_lengths_stress = torch.LongTensor([stn_stress.size(0)])
x_lengths_after = torch.LongTensor([stn_after.size(0)]) if stn_after is not None else None


In [ ]:
with torch.no_grad():
    # Before word
    audio_before = net_g.infer(
        stn_before.unsqueeze(0).to(device),
        x_lengths_before.to(device),
        noise_scale=1.0,
        noise_scale_w=0.8,
        length_scale=1.0
    )[0][0,0].cpu().numpy()

    # Stress word
    audio_stress = net_g.infer_with_stress(
        stn_stress.unsqueeze(0).to(device),
        x_lengths_stress.to(device),
        noise_scale=1.0,
        noise_scale_w= 0.8,
        length_scale= 1,
        stress_indices=list(range(stn_stress.size(0))),  # all phonemes of the word
        stress_length_scale= 2# exaggerate duration
    )[0][0,0].cpu().numpy()

    # After word (if any)
    audio_after = None
    if stn_after is not None:
        audio_after = net_g.infer(
            stn_after.unsqueeze(0).to(device),
            x_lengths_after.to(device),
            noise_scale=1.0,
            noise_scale_w=0.8,
            length_scale=1.0
        )[0][0,0].cpu().numpy()


In [103]:
import numpy as np

final_audio = audio_before
final_audio = np.concatenate([final_audio, audio_stress])
if audio_after is not None:
    final_audio = np.concatenate([final_audio, audio_after])
import IPython.display as ipd
ipd.display(ipd.Audio(final_audio, rate=hps.data.sampling_rate))


## VCTK

In [13]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [14]:
device = torch.device("cpu")  # CPU-only

# Initialize model on CPU
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model
).to(device)
_ = net_g.eval()

# Load the pretrained VCTK checkpoint
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_vctk.pth"
_ = utils.load_checkpoint(checkpoint_path, net_g, None)


c:\Users\MIKKY\vits-main\vits-main\utils.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')


INFO:root:Loaded checkpoint 'C:\Users\MIKKY\vits-main\vits-main\pretrained_vctk.pth' (iteration 0)


In [15]:
# pick device
device = torch.device("cpu")

# convert text → tensor
stn_tst = get_text("Good evening everyone. How are You ? VITS is Awesome!", hps)

with torch.no_grad():
    # send to CPU
    x_tst = stn_tst.unsqueeze(0).to(device)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([4]).to(device)  # target speaker ID

    # inference
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        sid=sid,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().float().numpy()

# play audio
import IPython.display as ipd
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


### Voice Conversion

In [16]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'DUMMY2/p234/p234_071.wav'

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))